# Initilization

## Cluster Connection

In [ ]:
# Import ipython widgets
import json
import math
import os

import ipywidgets as widgets
import matplotlib.pyplot
import numpy as np
import rich
import pandas as pd

# Set up the environment.
import scipy.signal
from IPython.display import display
from ipywidgets import fixed, interact, interact_manual, interactive

from qblox_instruments import Cluster, PlugAndPlay, Pulsar
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# close all previous connections to the cluster
Cluster.close_all()

#device_name = "pingu_cluster"
device_name = "loki_cluster"
ip_address = '192.0.2.72'
# connect to the cluster and reset
cluster = Cluster(device_name, ip_address)
cluster.reset()
print(f"{device_name} connected at {ip_address}")

In [ ]:

cluster.identify()

In [ ]:
# Cluster.close_all()

cluster.reset()

In [ ]:
cluster.reboot()

## SPI

In [ ]:
from tergite_autocalibration.utils.hardware_utils import SpiDAC
from tergite_autocalibration.config.coupler_config import coupler_spi_map
try:
    spi = SpiDAC()
except:
    pass
couplers = list(coupler_spi_map.keys())
dacs = [spi.create_spi_dac(coupler) for coupler in couplers]
coupler_dac = dict(zip(couplers,dacs))
print(couplers)
print([dac.current() for dac in dacs])

In [ ]:
parking_current = 0e-6
for coupler in couplers:
# coupler = couplers[0]
    dac = coupler_dac[coupler]
    spi.set_dac_current(dac, parking_current)
    print(f'Coupler {coupler} is parked at {dac.current()}.')

In [ ]:
coupler = 'q22_q23'
parking_current = 0e-6
spi.set_dac_current(coupler_dac[coupler], parking_current)
print(f'Coupler {coupler} is parked at {coupler_dac[coupler].current()}.')

In [ ]:
coupler = 'q18_q19'
parking_current = 1275e-6
spi.set_dac_current(coupler_dac[coupler], parking_current)
print(f'Coupler {coupler} is parked at {coupler_dac[coupler].current()}.')

In [ ]:
coupler = 'q19_q20'
parking_current = -825e-6
spi.set_dac_current(coupler_dac[coupler], parking_current)
print(f'Coupler {coupler} is parked at {coupler_dac[coupler].current()}.')

In [ ]:
coupler = 'q19_q24'
parking_current = -675e-6
spi.set_dac_current(coupler_dac[coupler], parking_current)
print(f'Coupler {coupler} is parked at {coupler_dac[coupler].current()}.')

### Update redis - parking current

In [ ]:
import redis
redis_connection = redis.Redis(decode_responses=True)
couplers = ['q19_q20']
for coupler in couplers: 
    print(f"Coupler parameters {coupler}:")
    redis_config = redis_connection.hgetall(f"transmons:{coupler}")
    for key, value in redis_config.items():
        print(f"{key}: {value}")

In [ ]:
coupler = 'q19_q22'
transmon_parameter = 'parking_current'
for coupler in couplers: 
    redis_connection.hset(f"transmons:{coupler}", f"{transmon_parameter}",parking_current)
parking_current = float(redis_connection.hget(f'transmons:{coupler}', 'parking_current'))
print(parking_current)

# TAC Calibration

In [ ]:
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
couplers = cz.keys()
cz_new = {}
for coupler in couplers: 
    param = {}
    redis_config = redis_connection.hgetall(f"couplers:{coupler}")
    for key in ['parking_current','cz_pulse_frequency','cz_pulse_duration',
    'cz_pulse_amplitude','cz_dynamic_target','cz_dynamic_control','tqg_fidelity']:
        param[key] = float(redis_config[key])
    cz_new[coupler] = param
cz_new

In [ ]:
cz = {'q18_q19': {'parking_current': -0.001050,
  'cz_pulse_frequency': 488138888.88888884,
  'cz_pulse_duration': 2.6776276912856313e-07,
  'cz_pulse_amplitude': 0.32,
  'cz_dynamic_target': 0,
  'cz_dynamic_control': 0,
  'tqg_fidelity': 0},
 'q19_q20': {'parking_current': -0.000825,
  'cz_pulse_frequency': 488138888.88888884,
  'cz_pulse_duration': 2.6776276912856313e-07,
  'cz_pulse_amplitude': 0.32,
  'cz_dynamic_target': 21.51425808355293,
  'cz_dynamic_control': 81.71441567052358,
  'tqg_fidelity': 0.9446938719947331}}
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
couplers = cz.keys()
print(cz)
for coupler in couplers:
  for key, value in cz[coupler].items():
    print(key,value)
    redis_connection.hset(f"couplers:{coupler}", key,value)

In [ ]:
128/4

In [ ]:
# !acli node reset -n 'resonator_spectroscopy'
# !acli node reset -n 'qubit_01_spectroscopy'
# !acli node reset -n 'coupler_spectroscopy'
# !acli node reset -n 'rabi_oscillations'
# !acli node reset -n 'ramsey_correction'
# # !acli node reset -n 'T1'
# # !acli node reset -n 'T2'
# # !acli node reset -n 'T2_echo'
# !acli node reset -n 'motzoi_parameter'
# !acli node reset -n 'n_rabi_oscillations'
# !acli node reset -n 'resonator_spectroscopy_1'
# # # !acli node reset -n 'randomized_benchmarking'
!acli node reset -n 'qubit_12_spectroscopy'
!acli node reset -n 'rabi_oscillations_12'
!acli node reset -n 'ramsey_correction_12'
!acli node reset -n 'resonator_spectroscopy_2'
# !acli node reset -n 'cz_chevron'
# !acli node reset -n 'cz_chevron_amplitude'
# !acli node reset -n 'cz_calibration'
# !acli node reset -n 'cz_dynamic_phase'
# !acli node reset -n 'cz_dynamic_phase_swap'
# !acli node reset -n 'randomized_benchmarking'
# !acli node reset -n 'tqg_randomized_benchmarking'
# !acli node reset -n 'tqg_randomized_benchmarking_interleaved'

In [ ]:
!acli calibration start

In [ ]:
from ipaddress import ip_address, IPv4Address

from tergite_autocalibration.config.settings import CLUSTER_IP
from tergite_autocalibration.scripts.calibration_supervisor import CalibrationSupervisor
from tergite_autocalibration.scripts.db_backend_update import update_mss
from tergite_autocalibration.utils.enums import MeasurementMode

cluster_mode: 'MeasurementMode' = MeasurementMode.real
parsed_cluster_ip: 'IPv4Address' = CLUSTER_IP
supervisor = CalibrationSupervisor(cluster_mode=cluster_mode,
                                    cluster_ip=parsed_cluster_ip)
supervisor.calibrate_system()

# TAC Sweep

## Reset

In [ ]:
from config_TL import Monitor
monitor = Monitor()
monitor.node_status('reset_calibration_ssro')

In [ ]:
all_results = []
for rep in range(50):
    print('++++++++++++++++++++++++++++++++++++++++++++')
    print(f'{rep = }')
    print('++++++++++++++++++++++++++++++++++++++++++++')
    monitor.calibrate_node('reset_calibration_ssro')
    all_results.append({'name':monitor.get_node_name(),'rep':rep,'result':monitor.get_results()})
    with open('notebooks/reset/reset_ssro/'+all_results[0]['name']+'_all_results.py', 'w') as f:
        f.write(f"all_results = {all_results}\n")

In [ ]:
# reset_data_rep = np.array([[0.    ,0.    ,0.    ,0.    ,0.    ,0.    ,0.    ,0.    ,0.   ],
# [0.    ,0.   , 0.   , 0.006 ,0.    ,0.    ,0.028 ,0.046 ,0.02 ],
# [0.    ,0.    ,0.    ,0.003 ,0.    ,0.  ,  0. ,   0.  ,  0.031],
# [0.000e+00, 0.000e+00 ,0.000e+00 ,0.000e+00 ,0.000e+00 ,0.000e+00, 9.266e-04 ,1.460e-03 ,0.000e+00],
# [0.    ,0.    ,0.   , 0. ,   0.  ,  0. ,   0.011 ,0. ,   0.   ],
# [0. ,   0.   , 0.  ,  0.   , 0.  ,  0.  ,  0.    ,0.008, 0.   ],
# [0. ,   0.  ,  0.   , 0.  ,  0.016, 0.   , 0.009, 0.  ,  0.02 ],
#  [0.  ,  0.,    0.   , 0.001, 0.    ,0.033 ,0.027 ,0.    ,0.   ],
#   [0.000e+00 ,9.692e-04 ,0.000e+00 ,0.000e+00 ,0.000e+00 ,2.502e-02, 5.597e-03 ,2.773e-03 ,2.308e-02]
# ])
# reset_fidelity_rep = 1-np.mean(reset_data_rep,axis=0)
# reset_std_rep = np.std(reset_data_rep,axis=0)
# print(reset_fidelity_rep)
# print(reset_std_rep)

In [ ]:
# reset_data = np.array([[0.004, 0.002, 0.    ,0.008,0.013,0.01 ,0.136,0.105,0.048],
# [0.    ,0.    ,0.    ,0.02  ,0.014 ,0.055 ,0.102 ,0.09  ,0.134],
# [0.    ,0.    ,0.    ,0.    ,0.001 ,0.    ,0.128 ,0.118 ,0.14 ],
# [0.   , 0.    ,0.    ,0.    ,0.    ,0.    ,0.051 ,0.041 ,0.087],
# [0.    ,0.    ,0.    ,0.    ,0.    ,0.007 ,0.109 ,0.078 ,0.082],
#  [0.009, 0.009 ,0.    ,0.    ,0.    ,0.    ,0.126 ,0.147 ,0.099],
#  [0.    ,0.    ,0.006 ,0.022 ,0.052 ,0.047 ,0.197 ,0.175 ,0.137],
#  [0.    ,0.016 ,0.    ,0.011 ,0.    ,0.019 ,0.127 ,0.101 ,0.085],
#  [0.    ,0.   , 0.    ,0.    ,0.    ,0.    ,0.065 ,0.061 ,0.086],
#   [0.000e+00 ,0.000e+00 ,0.000e+00 ,0.000e+00 ,2.158e-02 ,8.423e-04 ,1.211e-01 ,1.309e-01, 1.487e-01],
#  [0.    ,0.    ,0.   , 0.    ,0.    ,0.   , 0.025, 0.075 ,0.059] ])
# reset_fidelity = 1-np.mean(reset_data,axis=0)
# reset_std = np.std(reset_data,axis=0)
# print(reset_fidelity)
# print(reset_std)

## Chevron Bias point sweep

In [ ]:
from tergite_autocalibration.scripts.monitor import Monitor
import numpy as np
import os
monitor = Monitor()
monitor.node_status('cz_chevron')
# monitor.calibrate_node('cz_chevron')
current_sweep = (np.arange(-655,-750-1,-5))
# amp_sweep = np.arange(0.05,0.15+0.001,0.05)
# amp_sweep = [0.015,0.05,0.1,0.15]
amp_sweep = [100*1e-9]
print(f'{current_sweep = }')
# print(f'{amp_sweep = }')
current_sweep = current_sweep *1e-6
time = len(current_sweep)*len(amp_sweep)*6/60*2
print(f'{time = } hours')

In [ ]:
import numpy as np
from tergite_autocalibration.utils.hardware_utils import SpiDAC
from tergite_autocalibration.config.coupler_config import coupler_spi_map
try:
    spi = SpiDAC()
except:
    pass
all_couplers = list(coupler_spi_map.keys())
dacs = [spi.create_spi_dac(coupler) for coupler in all_couplers]
coupler_dac = dict(zip(all_couplers,dacs))
print(f'{all_couplers = }')
print([dac.current() for dac in dacs])

from tergite_autocalibration.utils.user_input import qubits,couplers
print(f'curent = {coupler_dac[couplers[0]].current()}')

In [ ]:
# parking_current = 0e-6
# for coupler in all_couplers:
#     dac = coupler_dac[coupler]
#     spi.set_dac_current(dac, parking_current)
#     print(f'Coupler {coupler} is parked at {dac.current()}.')

In [ ]:
all_results = []
for parking_current in current_sweep:
    print('--------------------------------------------')
    print(f'{parking_current = }')
    print('--------------------------------------------')
    spi.set_dac_current(coupler_dac[couplers[0]], parking_current)
    # monitor.calibrate_node('qubit_01_spectroscopy')
    # monitor.calibrate_node('rabi_oscillations')
    for amp in amp_sweep:
        print('++++++++++++++++++++++++++++++++++++++++++++')
        print(f'{amp = }')
        print('++++++++++++++++++++++++++++++++++++++++++++')
        # monitor.calibrate_node('cz_chevron',cz_pulse_amplitude = amp)
        # all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'amp':amp,'result':monitor.get_results()})
        #monitor.calibrate_node('cz_chevron',cz_pulse_amplitude = 0.15)
        #all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'amplitude':0.15,'result':monitor.get_results()})
        monitor.calibrate_node('cz_chevron_amplitude',cz_pulse_duration = amp)
        all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'duration':amp,'result':monitor.get_results()})
        date = all_results[0]['name'][:8]
        path = 'tergite_autocalibration/notebooks/cz/'+date+'/'
        try:
            os.mkdir(path)
        except:
            pass
        with open(path+couplers[0]+'_'+all_results[0]['name']+'_all_results.py', 'w') as f:
            f.write(f"all_results = {all_results}\n")

In [ ]:
from tergite_autocalibration.scripts.monitor import Monitor
import numpy as np
import os
monitor = Monitor()
monitor.node_status('cz_chevron')

import numpy as np
from tergite_autocalibration.utils.hardware_utils import SpiDAC
from tergite_autocalibration.config.coupler_config import coupler_spi_map
try:
    spi = SpiDAC()
except:
    pass
all_couplers = list(coupler_spi_map.keys())
dacs = [spi.create_spi_dac(coupler) for coupler in all_couplers]
coupler_dac = dict(zip(all_couplers,dacs))
print(f'{all_couplers = }')
print([dac.current() for dac in dacs])

from tergite_autocalibration.utils.user_input import qubits,couplers
print(f'curent = {coupler_dac[couplers[0]].current()}')

In [ ]:
all_results = []
current_sweep = [-725e-6]
cz_duration = 100e-9
amp_sweep = [0.15]
for parking_current in current_sweep:
    print('--------------------------------------------')
    print(f'{parking_current = }')
    print('--------------------------------------------')
    spi.set_dac_current(coupler_dac[couplers[0]], parking_current)
    # monitor.calibrate_node('qubit_01_spectroscopy')
    # monitor.calibrate_node('rabi_oscillations')
    for amp in amp_sweep:
        print('++++++++++++++++++++++++++++++++++++++++++++')
        print(f'{amp = }')
        print('++++++++++++++++++++++++++++++++++++++++++++')
        monitor.calibrate_node('cz_chevron',cz_pulse_amplitude = amp)
        all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'amplitude':0.15,'result':monitor.get_results()})
        monitor.calibrate_node('cz_chevron_amplitude',cz_pulse_duration = cz_duration)
        all_results.append({'name':monitor.get_name(),'parking_current':parking_current,'duration':amp,'result':monitor.get_results()})
        date = all_results[0]['name'][:8]
        path = 'tergite_autocalibration/notebooks/cz/'+date+'/'
        try:
            os.mkdir(path)
        except:
            pass
        with open(path+couplers[0]+'_'+all_results[0]['name']+'_all_results.py', 'w') as f:
            f.write(f"all_results = {all_results}\n")

In [6]:
cz = {'q14_q15': {'parking_current': -0.000735,
  'cz_pulse_frequency': 558168000.0,
  'cz_pulse_duration': 2.2e-07,
  'cz_pulse_amplitude': 0.13,
  'cz_dynamic_target': 0,
  'cz_dynamic_control': 0,
  'tqg_fidelity': 0}}

from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
print(cz)
for coupler in couplers:
    for key, value in cz[coupler].items():
        redis_connection.hset(f"couplers:{coupler}", key,value)

{'q14_q15': {'parking_current': -0.000735, 'cz_pulse_frequency': 558168000.0, 'cz_pulse_duration': 2.2e-07, 'cz_pulse_amplitude': 0.13, 'cz_dynamic_target': 0, 'cz_dynamic_control': 0, 'tqg_fidelity': 0}}


# TAC Optimization

## CMA-ES Optimization Reset

In [3]:
# best_params = optimize.best_params
best_params = {'reset_qc_duration': -195.0,
 'reset_qc_amplitude': -0.022000000000000002,
 'reset_cr_duration': -2.0,
 'reset_cr_amplitude': -0.006999999999999999}

In [4]:
reset_duration_qc = 5.089999999999999e-07
reset_amplitude_qc = 0.073
reset_duration_cr = 9.02e-07
reset_amplitude_cr = -0.0945

In [5]:
import numpy as np
times_qc = np.array([best_params['reset_qc_duration']])*1e-9
amps_qc = np.array([best_params['reset_qc_amplitude']])
times_cr = np.array([best_params['reset_cr_duration']])*1e-9
amps_cr = np.array([best_params['reset_cr_amplitude']])

In [ ]:
from utilities.user_input import qubits,couplers
from config_TL import Monitor
monitor = Monitor()
monitor.node_status('reset_calibration_ssro')
# monitor.calibrate_node('reset_calibration_ssro')
monitor.calibrate_node('reset_calibration_ssro',opt_reset_duration_qc = dict(zip(couplers,times_qc)),
                                                        opt_reset_amplitude_qc = dict(zip(couplers,amps_qc)),
                                                        opt_reset_duration_cr = dict(zip(couplers,times_cr)),
                                                        opt_reset_amplitude_cr = dict(zip(couplers,amps_cr)))

In [ ]:
from config_TL import Monitor
monitor = Monitor()
monitor.node_status('reset_calibration_ssro')
monitor.calibrate_node('reset_calibration_ssro')

In [ ]:
from utilities.reset_redis_node import ResetRedisNode
reset_redis = ResetRedisNode()
reset_redis.reset_node('reset_calibration_ssro')

In [ ]:
import redis
redis_connection = redis.Redis(decode_responses=True)

for qubit in ['q17','q22']: 
    print(f"Qubit parameters {qubit}:")
    redis_config = redis_connection.hgetall(f"transmons:{qubit}")
    for key, value in redis_config.items():
        print(f"{key}: {value}")

In [ ]:
couplers = ['q16_q21']
for coupler in couplers: 
    print(f"Coupler parameters {coupler}:")
    redis_config = redis_connection.hgetall(f"couplers:{coupler}")
    for key in best_params.keys():
        print(f"{key}: {redis_config[key]}")

In [ ]:
from utilities.user_input import qubits,couplers
from config_TL import Monitor
from utilities.reset_redis_node import ResetRedisNode
import numpy as np
import optuna
import arrow
import os

class OptimizeNode:
    def __init__(self, node):
        self.monitor = Monitor()
        self.reset_redis = ResetRedisNode()
        self.node = node
        self.qubits = qubits
        self.couplers = couplers
        sampler = optuna.samplers.CmaEsSampler(with_margin=True)
        self.study = optuna.create_study(sampler=sampler)

    def objective_cz(self,trial):

        times_qc = np.array([trial.suggest_float("reset_qc_duration", -10, 10,step=0.05)])*1e-9
        amps_qc = np.array([trial.suggest_float("reset_qc_amplitude", -0.05, 0.05,step=0.001)])
        # times_cr = np.array([trial.suggest_float("reset_cr_duration", -400, 400,step=1)])*1e-9
        # amps_cr = np.array([trial.suggest_float("reset_cr_amplitude", -0.05, 0.05,step=0.001)])
        # self.reset_redis.reset_node(self.node)
        self.monitor.calibrate_node('reset_calibration_ssro',opt_reset_duration_qc = dict(zip(couplers,times_qc)),
                                                        opt_reset_amplitude_qc = dict(zip(couplers,amps_qc)),
                                                        # opt_reset_duration_cr = dict(zip(couplers,times_cr)),
                                                        # opt_reset_amplitude_cr = dict(zip(couplers,amps_cr))
                                                        )
        results = self.monitor.get_results()
        all_results = [results[qubit] for qubit in ['q17','q22']]
        print(all_results)
        all_costs = [ 
            # 18*(1-res['reset_fidelity'])**2 + 
            res['reset_leakage']**2 for res in all_results]
        return sum(all_costs)

    def optimize_node(self):
        print(f"Optimizing {self.node}")
        self.study.optimize(self.objective_cz, n_trials=50)
        self.best_params = self.study.best_params
        print(f"Validating trail {self.study.best_trial.number} with params {self.best_params}")
        self.validate_cz()
        print(f"Optimization finished for {self.node}")
        return self.study
    
    def plot_optimization(self):
        return optuna.visualization.plot_optimization_history(self.study)

    def validate_cz(self,best_params = None):

        if best_params is None:
            best_params = self.best_params

        times_qc = np.array([best_params['reset_qc_duration']])*1e-9
        amps_qc = np.array([best_params['reset_qc_amplitude']])
        # times_cr = np.array([best_params['reset_cr_duration']])*1e-9
        # amps_cr = np.array([best_params['reset_cr_amplitude']])
        self.monitor.calibrate_node('reset_calibration_ssro',opt_reset_duration_qc = dict(zip(couplers,times_qc)),
                                                        opt_reset_amplitude_qc = dict(zip(couplers,amps_qc)),
                                                        # opt_reset_duration_cr = dict(zip(couplers,times_cr)),
                                                        # opt_reset_amplitude_cr = dict(zip(couplers,amps_cr))
                                                        )
        name = self.monitor.get_node_name()
        results = self.monitor.get_results()
        print('##########################################')
        print(f'Validation results from trial {self.study.best_trial.number}:')
        print(f'{name=}')
        print(f'{best_params=}')
        print(f'{results=}')
        print('##########################################')

        date = arrow.now().format('YYYYMMDDHHMMSS')
        path = 'notebooks/reset/'+date+'/'
        try:
            os.mkdir(path)
        except:
            pass
        
        all_results = []
        all_results.append({'name':name,'best_params':best_params, 'result':results})
        with open(path+all_results[0]['name']+'_reset_optimization.py', 'w') as f:
            f.write(f"all_results = {all_results}\n")
        
        fig = self.plot_optimization()
        fig.write_image(path+all_results[0]['name']+'_reset_calibration_optimization_history.png')
        fig.show()    
        return results
        

In [ ]:
optimize = OptimizeNode('reset_calibration_ssro')
optimize_results = optimize.optimize_node()

In [ ]:
optimize.validate_cz()

In [ ]:
print(optimize_results.best_trial.number)
print(optimize_results.best_trial,optimize_results.best_params)
optuna.visualization.plot_optimization_history(optimize_results)

In [ ]:
optimize = OptimizeNode('reset_calibration_ssro')
optimize_results = optimize.optimize_node()

In [ ]:
print(optimize_results.best_trial.number)
print(optimize_results.best_trial,optimize_results.best_params)
optuna.visualization.plot_optimization_history(optimize_results)

In [ ]:
optimize = OptimizeNode('reset_calibration_ssro')
optimize_results = optimize.optimize_node()

In [ ]:
optimize_results.best_trial.number

In [ ]:
reset_duration_qc = 704e-09
reset_amplitude_qc = 9.5e-2

In [ ]:
optimize_results.best_trial,optimize_results.best_params

In [ ]:
fig = optuna.visualization.plot_optimization_history(optimize_results)
fig.write_image('notebooks/reset_calibration_ssro_optimization_history.png')
fig.show()

In [ ]:
# optimize_results.get_trials()[60].params

In [ ]:
best_params = optimize_results.best_params
# best_params = {'freq': 0.20000000000000018, 'time': -16.0, 'amp': 0.07}
# best_params = optimize_results.get_trials()[60].params
# best_params = {'freq': 0.030000000000000027, 'time': 0.3000000000000007, 'amp': -0.002}
freqs = np.array([best_params['freq']])*1e6
times = np.array([best_params['time']])*1e-9
amps = np.array([best_params['amp']])

In [ ]:
# cz_pulse_frequency = {'q21_q22': 216060088.58646688}
# cz_pulse_duration = {'q21_q22': 3.8722015076688495e-07}
# cz_pulse_amplitude = {'q21_q22': 0.155}
# cz_pulse_frequency = {'q16_q21': 348030000.0}
# cz_pulse_duration = {'q16_q21': 4.0259066648795153e-07}
# cz_pulse_amplitude = {'q16_q21': 0.148}

In [ ]:
reset_redis.reset_node('cz_calibration_ssro')
monitor.calibrate_node('cz_calibration_ssro',opt_cz_pulse_frequency = dict(zip(couplers,freqs)), 
                                                        opt_cz_pulse_duration = dict(zip(couplers,times)),
                                                        opt_cz_pulse_amplitude = dict(zip(couplers,amps)))
monitor.get_results()

## CMA-ES Optimization CZ

In [7]:
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
cz,param = {},{}
for coupler in couplers: 
    redis_config = redis_connection.hgetall(f"couplers:{coupler}")
    for key in ['parking_current','cz_pulse_frequency','cz_pulse_duration',
    'cz_pulse_amplitude','cz_dynamic_target','cz_dynamic_control','tqg_fidelity']:
        try:
            param[key] = float(redis_config[key])
        except:
            pass
    cz[coupler] = param
cz

{'q14_q15': {'parking_current': -0.000735,
  'cz_pulse_frequency': 558168000.0,
  'cz_pulse_duration': 2.2e-07,
  'cz_pulse_amplitude': 0.13,
  'cz_dynamic_target': 0.0,
  'cz_dynamic_control': 0.0,
  'tqg_fidelity': 0.0}}

In [8]:
from tergite_autocalibration.scripts.monitor import Monitor, OptimizeNode
import numpy as np
import os
monitor = Monitor()
monitor.node_status('cz_calibration')
# monitor.calibrate_node('cz_calibration')

Qubits:
    q14: cz_calibration: calibrated
    q15: cz_calibration: not_calibrated
-----------------------
Couplers:
    q14_q15: cz_calibration: calibrated


In [9]:
optimize = OptimizeNode('cz_calibration',trails = 200, optimize_swap=True)
optimize_results = optimize.optimize_node()

/home/oarm/miniforge3/envs/autocalib/lib/python3.9/site-packages/optuna/samplers/_cmaes.py:313: ExperimentalWarning: `with_margin` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-06-20 01:14:05,128] A new study created in memory with name: no-name-9c48863b-749e-46e6-92eb-af9cdf092d1d


Optimizing cz_calibration with 200 trails
Optimizing cz_calibration with {'q14_q15': 3130000.0}, {'q14_q15': -1.8000000000000002e-08}, {'q14_q15': 0.026600000000000006}


2024-06-20 01:14:05,136 ◆ calibration_supervisor.py ◆ Calibrating node cz_calibration_ssro


these are all couplers:  {'q14_q15': <CompositeSquareEdge: q14_q15>}
these are all qubits:  [['q14', 'q15']]
cz_pulse_frequency = {'q14_q15': 561298000.0}
cz_pulse_duration = {'q14_q15': 2.02e-07}
cz_pulse_amplitude = {'q14_q15': 0.15660000000000002}
aa091ebb-3c76-4718-88f5-ac061ed3c1ff


2024-06-20 01:14:05,322 ◆ compilation_worker.py ◆ Starting Compiling


...

2024-06-20 07:37:59,064 ◆ execution_worker.py ◆ Finished measurement
2024-06-20 07:37:59,066 ◆ monitor_worker.py ◆ measurement completed


measurement completed


/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_phase' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_pop_loss' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_leakage' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_phase' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-ac

Validating trail 166 with params {'cz_pulse_frequency': -0.137, 'cz_pulse_duration': -9.0, 'cz_pulse_amplitude': 0.021200000000000004}


KeyError: 'cz_calibrate_ssro'

In [10]:
results = optimize.validate_cz()

KeyError: 'cz_calibrate_ssro'

In [ ]:
optimize_results.best_trial,optimize_results.best_params

In [ ]:
import optuna
optuna.visualization.plot_optimization_history(optimize_results)

## Update redis - CZ

In [9]:
cz = {'q14_q15': {'parking_current': -0.000735,
  'cz_pulse_frequency': 557985000.0,
  'cz_pulse_duration': 2.13e-07,
  'cz_pulse_amplitude': 0.151,
  'cz_dynamic_target': 0,
  'cz_dynamic_control': 0,
  'tqg_fidelity': 0}}

from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
print(cz)
couplers = ['q14_q15']
for coupler in couplers:
    for key, value in cz[coupler].items():
        redis_connection.hset(f"couplers:{coupler}", key,value)

{'q14_q15': {'parking_current': -0.000735, 'cz_pulse_frequency': 557985000.0, 'cz_pulse_duration': 2.13e-07, 'cz_pulse_amplitude': 0.151, 'cz_dynamic_target': 0, 'cz_dynamic_control': 0, 'tqg_fidelity': 0}}


In [3]:
best_params = optimize_results.best_params
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)

unit = {'cz_pulse_frequency': 1e6,'cz_pulse_duration': 1e-9,'cz_pulse_amplitude': 1}
for coupler in couplers:
    for key in best_params.keys():
        redis_connection.hset(f"couplers:{coupler}", f"{key}",cz[coupler][key]+best_params[key]*unit[key])

NameError: name 'optimize_results' is not defined

In [3]:
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
couplers = ['q14_q15']
redis_connection = redis.Redis(decode_responses=True)
cz_updated = {}
for coupler in couplers: 
    redis_config = redis_connection.hgetall(f"couplers:{coupler}")
    param = {}
    for key in ['parking_current','cz_pulse_frequency','cz_pulse_duration',
    'cz_pulse_amplitude','cz_dynamic_target','cz_dynamic_control','tqg_fidelity']:
        try:
            param[key] = float(redis_config[key])
        except:
            pass
    cz_updated[coupler] = param
print(f'{cz_updated = }')

cz_updated = {'q14_q15': {'parking_current': -0.000735, 'cz_pulse_frequency': 557985000.0, 'cz_pulse_duration': 2.13e-07, 'cz_pulse_amplitude': 0.151, 'cz_dynamic_target': 0.0, 'cz_dynamic_control': 0.0, 'tqg_fidelity': 0.0}}


In [ ]:
# rewrite the VNA_values.py file
with open('tergite_autocalibration/config/coupler_values.py', 'w+') as f:
    f.write(f"cz_values = {cz_updated}\n")

## Run RB

In [7]:
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
redis_connection = redis.Redis(decode_responses=True)
cz_new,param = {},{}
for coupler in couplers: 
    redis_config = redis_connection.hgetall(f"couplers:{coupler}")
    for key in ['parking_current','cz_pulse_frequency','cz_pulse_duration',
    'cz_pulse_amplitude','cz_dynamic_target','cz_dynamic_control','tqg_fidelity']:
        try:
            param[key] = float(redis_config[key])
        except:
            pass
    cz_new[coupler] = param
cz_new

{'q14_q15': {'parking_current': -0.000735,
  'cz_pulse_frequency': 507985000.0,
  'cz_pulse_duration': 2.13e-07,
  'cz_pulse_amplitude': 0.151,
  'cz_dynamic_target': 0.0,
  'cz_dynamic_control': 0.0,
  'tqg_fidelity': 0.0}}

In [ ]:
# {'q19_q20': {'parking_current': -0.000825,
#   'cz_pulse_frequency': 481058000.0,
#   'cz_pulse_duration': 2.2021909035268708e-07,
#   'cz_pulse_amplitude': 0.19435000000000002,
#   'cz_dynamic_target': 21.51425808355293,
#   'cz_dynamic_control': 81.71441567052358,
#   'tqg_fidelity': 0.9446938719947331}}

In [ ]:
#!acli node reset -n 'cz_chevron'
!acli node reset -n 'cz_calibration'
!acli node reset -n 'cz_dynamic_phase'
!acli node reset -n 'cz_dynamic_phase_swap'
# !acli node reset -n 'randomized_benchmarking'
!acli node reset -n 'tqg_randomized_benchmarking'
!acli node reset -n 'tqg_randomized_benchmarking_interleaved

In [ ]:
!acli node reset -n 'tqg_randomized_benchmarking'

!acli calibration start

In [ ]:
from tergite_autocalibration.utils.user_input import qubits,couplers
import redis
import numpy as np
redis_connection = redis.Redis(decode_responses=True)
couplers = ['q14_q15']
rb_dynamic_phase = np.arange(-180.0, 180.0, 10)
gate_time = np.arange(-20, 20, 2)*1e-9
print(rb_dynamic_phase)
for gate_time_opt in gate_time:
    new_gate_time = 213*1e-9+gate_time_opt
    cz = {'q14_q15': {'parking_current': -0.000735,
    'cz_pulse_frequency': 557985000.0,
    'cz_pulse_duration': new_gate_time,
    'cz_pulse_amplitude': 0.151,
    'cz_dynamic_target': 0,
    'cz_dynamic_control': 0,
    'tqg_fidelity': 0}}

    for coupler in couplers:
        for key, value in cz[coupler].items():
            redis_connection.hset(f"couplers:{coupler}", key,value)

    !acli node reset -n 'tqg_randomized_benchmarking'

    !acli calibration start

In [8]:
from tergite_autocalibration.scripts.monitor import Monitor
import numpy as np
import os
monitor = Monitor()
monitor.node_status('cz_calibration_swap_ssro')
monitor.calibrate_node('cz_calibration_swap_ssro')

Qubits:
    q14: cz_calibration_swap_ssro: calibrated
    q15: cz_calibration_swap_ssro: not_calibrated
-----------------------
Couplers:
    q14_q15: cz_calibration_swap_ssro: calibrated


2024-06-20 09:12:59,949 ◆ calibration_supervisor.py ◆ Calibrating node cz_calibration_swap_ssro


swapping
these are all couplers:  {'q14_q15': <CompositeSquareEdge: q14_q15>}
these are all qubits:  [['q14', 'q15']]
cz_pulse_frequency = {'q14_q15': 507985000.0}
cz_pulse_duration = {'q14_q15': 2.13e-07}
cz_pulse_amplitude = {'q14_q15': 0.151}
89a80d13-02e1-4e0b-9954-58aa072362ab


2024-06-20 09:13:00,078 ◆ compilation_worker.py ◆ Starting Compiling


precompilation completed
schedule_duration = 0.05 sec


2024-06-20 09:13:00,202 ◆ execution_worker.py ◆ Starting measurement
cz_calibration_swap_ssro: 0it [00:00, ?it/s]
/home/oarm/miniforge3/envs/autocalib/lib/python3.9/site-packages/quantify_scheduler/instrument_coordinator/components/qblox.py:1246: FutureWarning: The format of acquisition data of looped measurements in APPEND mode will change in quantify-scheduler>=0.18.0
  warnings.warn(
2024-06-20 09:13:51,432 ◆ execution_worker.py ◆ Raw dataset acquired


node dimenstions are:  [2, 25]
node dimenstions are:  [2, 25]


2024-06-20 09:13:51,482 ◆ execution_worker.py ◆ Finished measurement
2024-06-20 09:13:51,483 ◆ monitor_worker.py ◆ measurement completed


measurement completed


/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_phase' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_pop_loss' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_leakage' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-acl/tergite_autocalibration/utils/convert.py:121: UserWarning: 'cz_phase' is not in mapped parameter list in utilities/standard_redis_storage.py. Please add appropriate parameter atributes in the map
  warnings.warn(
/home/oarm/repos/tergite-ac